# BepiColombo PWI EFD Pot: L1a QL -- 2025/8/10

In [ ]:
import copy
import datetime
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from spacepy import pycdf

In [ ]:
sys.path.append('./lib/')
import Bepi_PWI_EFD_pot_data as bepi_data
import Bepi_PWI_EFD_pot_lib  as bepi_pot
# import Bepi_PWI_EFD_cal_lib  as bepi_cal

# parameter set -- to be modified

In [ ]:
# *** Dump ***
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** CAL ***
unit_mode = 2                           # [Power]     0: RAW    1: V    2: V/m
band_mode = 0                           # [Power]     0: sum    1: /Hz
cal_mode  = 2                           # [Power]     0: background     1: CAL     2: all

# *** Unit ***
time_mode = 0                           # [Time]      0: Epoch   1: data number
gap_mode  = 1                           # [gap]       0: n/a     1: gap

# *** Power range set ***
p_raw_max = +10                         # background: 7.5   CAL: 10
p_raw_min = -10                         # background: 2.5   CAL: 5

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/Bepi_python/ql/'   # Plot dump folder

# get CDF data

In [ ]:
mode_cdf = 1        # 0:flight  1:test
mode_tlm = 'm'      # l, m, h
mode_ant = 0        # 0:both    1:U(WPT)    2:V(MEF)

date = '0'          # from Data list in Bepi_PWI_EFD_sec_data.py
# TEST
date = '20250801'       # Initial Check DryRun #3
# date = '20250625'     # Initial Check DryRun #2
# date = '2025060?'     # Initial Check DryRun #1
# date = '20250606'     # Initial Check DryRun #1
# date = '20250605'     # Initial Check DryRun #1
# date = '20250606'     # Initial Check DryRun #1
# date = '20250625'     # Initial Check DryRun #2

# Flight -- only 'l'
# date='20250410'
# date='20240409'
# date='20231106'
# date='20231026'           # WPT: CAL
# date='20230730'
# date='20230522'
# date='20230228'           # WPT: CAL
# date='20221107'
# date='20220517'
# date='20220311'
# date='20211124'
# date='20210617'
# date='20201217'
# date='20200304'
# date='20190630'
# date='2018080?'           # WPT latch release
# date='20190807'           # WPT latch release
# date='20190806'           # WPT latch release
# date='20190805'           # WPT precheck
# date='20190802'           # WPT precheck
# date='20190701'           # WPT detailed check
# date='20190515'
# date='201811[0-1][9,0]'   # Init check
# date='20181111'           # Init check
# date='20181110'           # Init check
# date='20181109'           # Init check

data_dir, data_list = bepi_data.datalist(date, mode_tlm, mode_cdf)       # [date]   yyyymmdd: group read    others: file list

In [ ]:
class struct:
    pass
data = struct()
num_list = len(data_list)
for i in range(num_list):
    data_name = data_list[i];   cdf_file = data_dir + data_name
    cdf = pycdf.CDF(cdf_file);  data1 = bepi_pot.efd_pot_read(cdf, mode_tlm)
    if i==0: data = data1;                               
    else:    data = bepi_pot.efd_pot_add(data, data1, mode_tlm)
    print(i, data.Vu1.shape, cdf_file)
data_name = os.path.split(data_name)[1];            print("data name:", data_name)

In [ ]:
data = bepi_pot.efd_pot_shaping(data, cal_mode, mode_tlm, mode_ant)

In [ ]:
if mode_tlm=='l':
    print("         data.Vu1[V]:", data.Vu1.shape,             data.Vu1[0],             data.Vu1[-1])
    print("         data.Vu2[V]:", data.Vu2.shape,             data.Vu2[0],             data.Vu2[-1])
    print("         data.Vv1[V]:", data.Vv1.shape,             data.Vv1[0],             data.Vv1[-1])
    print("         data.Vv2[V]:", data.Vv2.shape,             data.Vv2[0],             data.Vv2[-1])
else:
    print("    data.Vu1(,dt)[V]:", data.Vu1.shape,             data.Vu1[0][0],          data.Vu1[-1][-1])
    print("    data.Vu2(,dt)[V]:", data.Vu2.shape,             data.Vu2[0][0],          data.Vu2[-1][-1])
    print("    data.Vv1(,dt)[V]:", data.Vv1.shape,             data.Vv1[0][0],          data.Vv1[-1][-1])
    print("    data.Vv2(,dt)[V]:", data.Vv2.shape,             data.Vv2[0][0],          data.Vv2[-1][-1])
if mode_tlm!='h':
    print("     data.PRE_U_OBS:", data.PRE_U_OBS.shape,     "\t", data.PRE_U_OBS[0],        data.PRE_U_OBS[-1])
    print("     data.PRE_V_OBS:", data.PRE_V_OBS.shape,     "\t", data.PRE_V_OBS[0],        data.PRE_V_OBS[-1])
    print("    data.PRE_U_ACAL:", data.PRE_U_ACAL.shape,    "\t", data.PRE_U_ACAL[0],       data.PRE_U_ACAL[-1])
    print("       data.EFD_CAL:", data.EFD_CAL.shape,       "\t", data.EFD_CAL[0],          data.EFD_CAL[-1])
    print("        data.BIAS_U:", data.BIAS_U.shape,        "\t", data.BIAS_U[0],           data.BIAS_U[-1])
    print("        data.BIAS_V:", data.BIAS_V.shape,        "\t", data.BIAS_V[0],           data.BIAS_V[-1])
    print("      data.AM2P_ACT:", data.AM2P_ACT.shape,      "\t", data.AM2P_ACT[0],         data.AM2P_ACT[-1])
    print("     data.EFD_Hdump:", data.EFD_Hdump.shape,     "\t", data.EFD_Hdump[0],        data.EFD_Hdump[-1])
    print("     data.EFD_U_ENA:", data.EFD_U_ENA.shape,     "\t", data.EFD_U_ENA[0],        data.EFD_U_ENA[-1])
    print("     data.EFD_V_ENA:", data.EFD_V_ENA.shape,     "\t", data.EFD_V_ENA[0],        data.EFD_V_ENA[-1])
    #
    print("    data.BIAS_LVL_U1:", data.BIAS_LVL_U1.shape,   "\t", data.BIAS_LVL_U1[0],     data.BIAS_LVL_U1[-1])
    print("    data.BIAS_LVL_U2:", data.BIAS_LVL_U2.shape,   "\t", data.BIAS_LVL_U2[0],     data.BIAS_LVL_U2[-1])
    print("    data.BIAS_LVL_V1:", data.BIAS_LVL_V1.shape,   "\t", data.BIAS_LVL_V1[0],     data.BIAS_LVL_V1[-1])
    print("    data.BIAS_LVL_V2:", data.BIAS_LVL_V2.shape,   "\t", data.BIAS_LVL_V2[0],     data.BIAS_LVL_V2[-1])
    print("    data.HK_WPT:",      data.HK_WPT.shape,        "\t", data.HK_WPT[0],          data.HK_WPT[-1])
    print("    data.HK_MEF:",      data.HK_MEF.shape,        "\t", data.HK_MEF[0],          data.HK_MEF[-1])
print(    " data.EFD_saturation:", data.EFD_saturation.shape,"\t", data.EFD_saturation[0],  data.EFD_saturation[-1])
print(    "   data.EFD_spinrate:", data.EFD_spinrate.shape,  "\t", data.EFD_spinrate[0],    data.EFD_spinrate[-1])
print(    "  data.EFD_spinphase:", data.EFD_spinphase.shape, "\t", data.EFD_spinphase[0],   data.EFD_spinphase[-1])
print(    "          data.epoch:", data.epoch.shape,         "\t", data.epoch[0],           data.epoch[-1])
print(    "         data.EFD_TI:", data.EFD_TI.shape,        "\t", '{:X}'.format(data.EFD_TI[0]), '{:X}'.format(data.EFD_TI[-1]))

In [ ]:
date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
n_time0 = data.n_time

if data.n_time>1: 
    print("    Epoch:", str_date, "(interval:", data.epoch[1] - data.epoch[0], data.epoch[-1] - data.epoch[-2], ")")
else:
    print("    Epoch:", str_date)
print("  Samples:", n_time0)

In [ ]:
pot = copy.deepcopy(data)

### M data -- time resolution up

In [ ]:
from datetime import timedelta
Epoch_1d = pot.epoch.tolist();   Epoch_pot = []
if (data.n_dt > 1):
    for i in range(data.n_dt):
        Epoch_pot += Epoch_1d
    for i in range(n_time0):
        for j in range(data.n_dt):
            Epoch_pot[i*data.n_dt + j] = Epoch_1d[i] + timedelta(seconds = j/data.n_dt)
else:
    Epoch_pot += Epoch_1d
n_time1 = n_time0 * data.n_dt; num_1d = np.arange(n_time1)

In [ ]:
# pot = bepi_cal.pot_cal(pot)     # CAL
pot.cf       = 0.0
pot.str_unit = "[V]"

### Peak

In [ ]:
bepi_pot.pot_peak(pot, n_time0)

In [ ]:
p_max0 = p_raw_max + pot.cf/10;  p_min0 = p_raw_min + pot.cf/10
p_max0  = np.ceil( np.nanmax( [np.nanmax(pot.Vu1), np.nanmax(pot.Vu2), np.nanmax(pot.Vv1), np.nanmax(pot.Vv2)] )*5)/5 + 1
p_min0  = np.ceil( np.nanmin( [np.nanmin(pot.Vu1), np.nanmin(pot.Vu2), np.nanmin(pot.Vv1), np.nanmin(pot.Vv2)] )*5)/5 - 1
print("conversion factor:", '{:.1f}'.format(pot.cf), "    MAX-min:", '{:.1f}'.format(p_max0), '{:.1f}'.format(p_min0), "   str_unit:", pot.str_unit)

In [ ]:
if   mode_tlm=='l':     # L
    str_Vu1 = 'Vu1_1Hz (L)';    str_Vu2 = 'Vu2_1Hz (L)';    str_Vv1 = 'Vv1_1Hz (L)';    str_Vv2 = 'Vv2_1Hz (L)';    str_V = 'V_1Hz (L)'
elif mode_tlm=='m':     # M
    str_Vu1 = 'Vu1_8Hz (M)';    str_Vu2 = 'Vu2_8Hz (M)';    str_Vv1 = 'Vv1_8Hz (M)';    str_Vv2 = 'Vv2_8Hz (M)';    str_V = 'V_8Hz (M)'
else:                   # H
    str_Vu1 = 'Vu1_32Hz (H)';   str_Vu2 = 'Vu2_32Hz (H)';   str_Vv1 = 'Vv1_32Hz (H)';   str_Vv2 = 'Vv2_32Hz (H)';   str_V = 'V_32Hz (H)'


In [ ]:
# NAN
if gap_mode == 1 and time_mode == 0:
    for i in range(n_time0-1):
        dt = pot.epoch[i+1] - pot.epoch[i]
        if dt.total_seconds() > 60:  bepi_pot.pot_nan(pot, i)
pot.V = (pot.Vu1 + pot.Vu2 + pot.Vv1 + pot.Vv2)/4

# RAW plot

In [ ]:
n0 = 0;                 n1    = n_time0
# n0 = n_time0//6-100;    n1    = n_time0//6-50
p_max  = p_max0+2;      p_min = p_min0

fig = plt.figure(figsize=(14, 11))
if mode_tlm!='h':
    ax1 = fig.add_subplot(4, 1, 1);  ax2 = fig.add_subplot(4, 1, 2);  ax3 = fig.add_subplot(4, 1, 3);  ax4 = fig.add_subplot(4, 1, 4)
else:
    ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)
ax1.plot(np.ravel(pot.V  [n0:n1]),                 '-y', linewidth=2.0, label=str_V)
ax1.plot(np.ravel(pot.Vu1[n0:n1])+4,               '-r', linewidth=1.0, label=str_Vu1 + ' +4')
ax1.plot(np.ravel(pot.Vu2[n0:n1])+3,               ':r', linewidth=0.4, label=str_Vu2 + ' +3')
ax1.plot(np.ravel(pot.Vv1[n0:n1])+2,               '-b', linewidth=1.0, label=str_Vv1 + ' +2')
ax1.plot(np.ravel(pot.Vv2[n0:n1])+1,               ':b', linewidth=0.4, label=str_Vv2 + ' +1')
if mode_tlm!='h':
    ax2.plot(np.ravel(data.PRE_U_OBS [n0:n1]) +20, '-r', linewidth=2.0, label='PRE_U_OBS')
    ax2.plot(np.ravel(data.PRE_V_OBS [n0:n1]) +18, '-b', linewidth=1.0, label='PRE_V_OBS')
    ax2.plot(np.ravel(data.PRE_U_ACAL[n0:n1]) +16, ':r', linewidth=2.0, label='PRE_U_ACAL')
    ax2.plot(np.ravel(data.EFD_CAL   [n0:n1]) +14, ':b', linewidth=1.0, label='EFD_CAL')
    ax2.plot(np.ravel(data.BIAS_U    [n0:n1]) +12, '-r', linewidth=2.0, label='BIAS_U')
    ax2.plot(np.ravel(data.BIAS_V    [n0:n1]) +10, '-b', linewidth=1.0, label='BIAS_V')
    ax2.plot(np.ravel(data.EFD_U_ENA [n0:n1])  +8, ':r', linewidth=2.0, label='EFD_U_ENA')
    ax2.plot(np.ravel(data.EFD_V_ENA [n0:n1])  +6, ':b', linewidth=1.0, label='EFD_V_ENA')
    ax2.plot(np.ravel(data.AM2P_ACT  [n0:n1])  +4, '-r', linewidth=2.0, label='AM2P_ACT')
    ax2.plot(np.ravel(data.EFD_Hdump [n0:n1])  +2, '-g', linewidth=1.0, label='EFD_Hdump')
ax2.plot(np.ravel(data.EFD_saturation[n0:n1]),     '-k', linewidth=3.0, label='EFD_saturation')
if mode_tlm!='h':
    ax3.plot(np.ravel(data.BIAS_LVL_U1[n0:n1]),    '-r', linewidth=0.4, label='BIAS_LVL_U1')    
    ax3.plot(np.ravel(data.BIAS_LVL_U2[n0:n1]),    ':r', linewidth=0.4, label='BIAS_LVL_U2')    
    ax3.plot(np.ravel(data.BIAS_LVL_V1[n0:n1]*10), '-b', linewidth=0.4, label='BIAS_LVL_V1 *10')
    ax3.plot(np.ravel(data.BIAS_LVL_V2[n0:n1]*10), ':b', linewidth=0.4, label='BIAS_LVL_V2 *10')
    # ax3.plot(np.ravel(data.EFD_spinrate[n0:n1]), '-g', linewidth=1.0, label='EFD_spinrate')
    # ax4.plot(np.ravel(data.EFD_spinphase[n0:n1]),'-r', linewidth=1.0, label='EFD_spinphase')
    ax4.plot(np.ravel(data.epoch[n0:n1]),          '.', markersize=8)
    ax4.set_xlabel(str_date)
else:
    ax3.plot(np.ravel(data.epoch[n0:n1]),          '.', markersize=8)
    ax3.set_xlabel(str_date)
#
ax1.set_ylabel(pot.str_unit)
title_label = '[BepiColombo PWI EFD Potential]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  ax3.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.Vu1[n0:n1])) -.5];    print(xlim); ax1.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch[n0:n1]))-.5];   print(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];                    ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

In [ ]:
n0    = 0;              n1    = n_time0
n0    = 15000;          n1    = 17000
n0e   = n0 * data.n_dt; n1e   = n1 * data.n_dt
p_max = p_max0+2;       p_min = p_min0

fig = plt.figure(figsize=(14, 11))
if mode_tlm!='h':
    ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)
else:
    ax1 = fig.add_subplot(2, 1, 1);  ax2 = fig.add_subplot(2, 1, 2)
ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.V[n0:n1]),                  '-y', linewidth=2.0, label=str_V)
ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vu1[n0:n1])+.4,             '-r', linewidth=1.0, label=str_Vu1 + ' +4')
ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vu2[n0:n1])+.3,             ':r', linewidth=0.4, label=str_Vu2 + ' +3')
ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vv1[n0:n1])+.2,             '-b', linewidth=1.0, label=str_Vv1 + ' +2')
ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vv2[n0:n1])+.1,             ':b', linewidth=0.4, label=str_Vv2 + ' +1')
if mode_tlm!='h':
    ax2.plot(data.epoch[n0:n1], np.ravel(data.PRE_U_OBS [n0:n1]) +20, '-r', linewidth=2.0, label='PRE_U_OBS')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.PRE_V_OBS [n0:n1]) +18, '-b', linewidth=1.0, label='PRE_V_OBS')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.PRE_U_ACAL[n0:n1]) +16, ':r', linewidth=2.0, label='PRE_U_ACAL')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.EFD_CAL   [n0:n1]) +14, ':b', linewidth=1.0, label='EFD_CAL')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.BIAS_U    [n0:n1]) +12, '-r', linewidth=2.0, label='BIAS_U')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.BIAS_V    [n0:n1]) +10, '-b', linewidth=1.0, label='BIAS_V')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.EFD_U_ENA [n0:n1])  +8, ':r', linewidth=2.0, label='EFD_U_ENA')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.EFD_V_ENA [n0:n1])  +6, ':b', linewidth=1.0, label='EFD_V_ENA')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.AM2P_ACT  [n0:n1])  +4, '-r', linewidth=2.0, label='AM2P_ACT')
    ax2.plot(data.epoch[n0:n1], np.ravel(data.EFD_Hdump [n0:n1])  +2, '-g', linewidth=1.0, label='EFD_Hdump')
ax2.plot(data.epoch[n0:n1], np.ravel(data.EFD_saturation[n0:n1]),     '-k', linewidth=3.0, label='EFD_saturation')
if mode_tlm!='h':
    ax3.plot(data.epoch[n0:n1], np.ravel(data.BIAS_LVL_U1[n0:n1]),    '-r', linewidth=0.4, label='BIAS_LVL_U1')
    ax3.plot(data.epoch[n0:n1], np.ravel(data.BIAS_LVL_U2[n0:n1]),    ':r', linewidth=0.4, label='BIAS_LVL_U2')
    ax3.plot(data.epoch[n0:n1], np.ravel(data.BIAS_LVL_V1[n0:n1]*10), '-b', linewidth=0.4, label='BIAS_LVL_V1 *10')
    ax3.plot(data.epoch[n0:n1], np.ravel(data.BIAS_LVL_V2[n0:n1]*10), ':b', linewidth=0.4, label='BIAS_LVL_V2 *10')
# ax3.plot(data.epoch[n0:n1], np.ravel(data.EFD_spinrate [n0:n1]),    '-g', linewidth=1.0, label='EFD_spinrate')
# ax4.plot(data.epoch[n0:n1], np.ravel(data.EFD_spinphase[n0:n1]),    '-r', linewidth=1.0, label='EFD_spinphase')

ax1.set_ylabel(pot.str_unit)
title_label = '[BepiColombo PWI EFD Potential]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8);  
if mode_tlm!='h':
    ax3.legend(loc='upper right', fontsize=8)
    ax3.set_xlabel(str_date)
else:
    ax2.set_xlabel(str_date)

xlim=[Epoch_pot[n0e], Epoch_pot[n1e-1]];    print(xlim); ax1.set_xlim(xlim)
xlim=[data.epoch[n0], data.epoch[n1-1]];    print(xlim); ax2.set_xlim(xlim); 
if mode_tlm!='h':
    ax3.set_xlim(xlim)
# ylim=[10**p_min, 10**p_max];                    ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw2.png'
    fig.savefig(png_fname)

In [ ]:
if mode_tlm!='h':
    print(data.PRE_U_OBS[n0:n0+30],     data.PRE_U_OBS[n1-30:n1], 'PRE_U_OBS')
    print(data.PRE_V_OBS[n0:n0+30],     data.PRE_V_OBS[n1-30:n1], 'PRE_V_OBS')
    print(data.PRE_U_ACAL[n0:n0+30],    data.PRE_U_ACAL[n1-30:n1],'PRE_U_ACAL')
    print(data.EFD_CAL[n0:n0+30],       data.EFD_CAL[n1-30:n1],   'EFD_CAL')
    print(data.BIAS_U[n0:n0+30],        data.BIAS_U[n1-30:n1],    'BIAS_U') 
    print(data.BIAS_V[n0:n0+30],        data.BIAS_V[n1-30:n1],    'BIAS_V') 
    print(data.EFD_U_ENA[n0:n0+30],     data.EFD_U_ENA[n1-30:n1], 'EFD_U_ENA')
    print(data.EFD_V_ENA[n0:n0+30],     data.EFD_V_ENA[n1-30:n1], 'EFD_V_ENA')
    print(data.AM2P_ACT[n0:n0+30],      data.AM2P_ACT[n1-30:n1],  'AM2P_ACT')
    print(data.EFD_Hdump[n0:n0+30],     data.EFD_Hdump[n1-30:n1], 'EFD_Hdump')
print(data.EFD_saturation[n0:n0+30],data.EFD_saturation[n1-30:n1], 'EFD_saturation')
if mode_tlm!='h':
    print(data.BIAS_LVL_U1[n0:n0+10], "\t", data.BIAS_LVL_U1[n1-10:n1], 'BIAS_LVL_U1')
    print(data.BIAS_LVL_U2[n0:n0+10], "\t", data.BIAS_LVL_U2[n1-10:n1], 'BIAS_LVL_U2') 
    print(data.BIAS_LVL_V1[n0:n0+10], "\t", data.BIAS_LVL_V1[n1-10:n1], 'BIAS_LVL_V1') 
    print(data.BIAS_LVL_V2[n0:n0+10], "\t", data.BIAS_LVL_V2[n1-10:n1], 'BIAS_LVL_V2') 
    print(data.HK_WPT[n0], data.HK_WPT[n0+1], "\t\t", data.HK_WPT[n1-2], data.HK_WPT[n1-1],  'HK_WPT') 
    print(data.HK_MEF[n0], data.HK_MEF[n0+1], "\t",   data.HK_MEF[n1-2], data.HK_MEF[n1-1],  'HK_MEF')

# Potential

In [ ]:
n0     = 0;                 n1     = n_time0
# n0     = n_time0//10;       n1     = n_time0//6
n0e    = n0 * data.n_dt;    n1e    = n1 * data.n_dt
p_max  = p_max0+2;          p_min  = p_min0

fig = plt.figure(figsize=(14, 11))
if time_mode == 1:
    ax1 = fig.add_subplot(2, 1, 1); ax2 = fig.add_subplot(2, 1, 2)
    ax1.plot(num_1d[n0e:n1e],    np.ravel(pot.V[n0:n1]),      '-y', linewidth=2.0, label=str_V)
    ax1.plot(num_1d[n0e:n1e],    np.ravel(pot.Vu1[n0:n1])-.1, '-r', linewidth=1.0, label=str_Vu1 + ' -.1')
    ax1.plot(num_1d[n0e:n1e],    np.ravel(pot.Vu2[n0:n1])-.2, ':r', linewidth=0.4, label=str_Vu2 + ' -.2')
    ax1.plot(num_1d[n0e:n1e],    np.ravel(pot.Vv1[n0:n1])-.3, '-b', linewidth=1.0, label=str_Vv1 + ' -.3')
    ax1.plot(num_1d[n0e:n1e],    np.ravel(pot.Vv2[n0:n1])-.4, ':b', linewidth=0.4, label=str_Vv2 + ' -.4')
    ax2.plot(num_1d[n0e:n1e],    Epoch_pot[n0e:n1e],   '-k', linewidth=2.0, label='Epoch')
    ax2.set_xlabel(str_date)
else:
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.V[n0:n1]),      '-y', linewidth=2.0, label=str_V)
    ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vu1[n0:n1])-.1, '-r', linewidth=1.0, label=str_Vu1 + ' -.1')
    ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vu2[n0:n1])-.2, ':r', linewidth=1.0, label=str_Vu2 + ' -.2')
    ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vv1[n0:n1])-.3, '-b', linewidth=1.0, label=str_Vv1 + ' -.3')
    ax1.plot(Epoch_pot[n0e:n1e], np.ravel(pot.Vv2[n0:n1])-.4, ':b', linewidth=1.0, label=str_Vv2 + ' -.4')
    ax1.set_xlabel(str_date)
ax1.set_ylabel(pot.str_unit)
#
title_label = '[BepiColombo PWI EFD Potential]  ' + data_name;  ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8)

if time_mode == 1:
    xlim=[num_1d[n0e], num_1d[n1e-1]]
    ax1.set_xlim(xlim); ax2.set_xlim(xlim)
else:
    xlim=[Epoch_pot[n0e], Epoch_pot[n1e-1]]
    #E_min = '2024-08-21 04:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    #E_max = '2024-08-21 19:00:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    ax1.set_xlim(xlim)
# ylim=[p_min, p_max];  ax1.set_ylim(ylim)

fig.subplots_adjust(hspace=0);  fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_V.png'
    fig.savefig(png_fname)